# Verifica delle fonti dati

Questo notebook permette di testare rapidamente il recupero dei dati dalle principali fonti configurate nel progetto PREACT.

> ℹ️ La configurazione completa include cinque connettori (`GDELT`, `ACLED`, `UNHCR`, `HDX`, `Economic_Indicators`). Qui concentriamo gli esempi sulle tre fonti più utilizzate nel flusso base (eventi da **GDELT**, conflitti da **ACLED** ed indicatori macro-economici dalla **World Bank**).

## Import dei moduli necessari

In [ ]:
from datetime import datetime
import os

import pandas as pd
from IPython.display import display

from preact.config import DataSourceConfig
from preact.data_ingestion.sources import build_sources, fetch_all

## Configurazione delle credenziali

Le API di ACLED possono essere consultate solo dopo registrazione, mentre alcune chiamate UNHCR/HDX richiedono un bearer token. Impostiamo qui dei segnaposto (`xxxxx`) così da avere ben chiari i parametri da sostituire con le chiavi reali ottenute in fase di registrazione.

In [ ]:
os.environ.setdefault("ACLED_API_TOKEN", "xxxxx")
os.environ.setdefault("UNHCR_API_TOKEN", "xxxxx")
os.environ.setdefault("HDX_API_TOKEN", "xxxxx")

## Definizione delle fonti da testare

Per ciascuna fonte definiamo il relativo `DataSourceConfig`. È possibile modificare facilmente parametri, intervalli temporali e chiavi prima di eseguire il notebook.

In [ ]:
source_configs = [
    DataSourceConfig(
        name="GDELT",
        endpoint="https://api.gdeltproject.org/api/v2/events",
    ),
    DataSourceConfig(
        name="ACLED",
        endpoint="https://api.acleddata.com/acled/read",
        requires_key=True,
        key_env_var="ACLED_API_TOKEN",
        key_param="key",
    ),
    DataSourceConfig(
        name="Economic_Indicators",
        endpoint="https://api.worldbank.org/v2/country/{country}/indicator/{indicator}",
        params={
            "country": "WLD",
            "indicators": "FP.CPI.TOTL.ZG,NY.GDP.MKTP.KD.ZG",
            "aliases": "{\"FP.CPI.TOTL.ZG\": \"inflation_rate\", \"NY.GDP.MKTP.KD.ZG\": \"gdp_growth\"}",
            "per_page": "2000",
        },
    ),
]
sources = build_sources(source_configs)

## Download dei dati di esempio

Di default vengono recuperati gli ultimi 14 giorni. È possibile modificare `lookback_days` o la data di fine (`end_date`) secondo necessità.

In [ ]:
lookback_days = 14
end_date = datetime.utcnow()

results = fetch_all(sources, lookback_days=lookback_days, end=end_date)

## Esplorazione rapida dei risultati

Per ogni fonte mostriamo le prime righe del dataset restituito e la relativa metainformazione di ingestione (timestamp, range temporale, uso di fallback ecc.).

In [ ]:
for name, result in results.items():
    if result is None:
        print(f"⚠️ {name}: nessun risultato disponibile")
        continue
    print(f"=== {name} ===")
    display(result.data.head())
    display(pd.DataFrame([result.metadata]))

## Prossimi passi

* Sostituire i valori `xxxxx` con le chiavi reali per ottenere dati autentici dalle API protette.
* Aumentare `lookback_days` per fetch più ampi oppure impostare `end_date` a una data storica.
* Salvare i dataframe con `result.data.to_parquet(...)` o `to_csv(...)` per analisi offline.